# DataBolt SmartJoin

Enhances `pd.merge()` function with:
* Pre- and post-join diagnostics
* Customize join type by join key
* Fuzzy top1 similarity joins for strings and dates
* Multi-frame joins
* Easier UI

In [1]:
import pandas as pd
import numpy as np

import join.smart_join

# Example 1: join 2 datasets with misalgined dates

In [2]:
df1=pd.read_csv('data/case_date_ma/df_cc_yoy_test2.csv',parse_dates=['delivery_date', 'reportenddate', 'reportstartdate'])
df2=pd.read_csv('data/case_date_ma/df_ma1.csv',parse_dates=['delivery_date', 'reportenddate'])
df2['reportstartdate']=df2['reportenddate'].shift(-1)


In [3]:
sj = join.smart_join.SmartJoin([df1, df2], ['delivery_date', 'reportstartdate','GICS_SUB_INDUSTRY_NAME'], mode=['top1 left', 'top1 left','exact inner'])

### Before you do a join, you often wonder what type of join you should use. With SmartJoin you can run prejoin disagnostics to see which of your join keys match and which don't.

We can see that only `GICS_SUB_INDUSTRY_NAME` has all values matched. The other two columns have unmatched dates on both sides.

In [4]:
sj.stats_prejoin(do_print=False)

,key left,key right,all matched,left,right,inner,outer,unmatched total,unmatched left,unmatched right
0,delivery_date,delivery_date,False,46,54,2,98,96,44,52
1,reportstartdate,reportstartdate,False,21,53,2,72,70,19,51
2,GICS_SUB_INDUSTRY_NAME,GICS_SUB_INDUSTRY_NAME,True,1,1,1,1,0,0,0
3,__all__,__all__,False,88,54,1,141,140,87,53


### Let's find the closest matching dates for `reportstartdate`

Since we said `reportstartdate` is a `top1 left` join, we can find the closest matching dates. 

In [5]:
r = sj.gen_match_tables(1)
r['table left']

,__left__,__right__,__diff__,__match type__
695,2015-10-01,2015-09-30,1 days,top1 left
747,2016-01-01,2015-12-31,1 days,top1 left
268,2016-02-18,2016-03-31,42 days,top1 left
586,2016-04-01,2016-03-31,1 days,top1 left
374,2016-05-08,2016-03-31,38 days,top1 left
321,2016-05-12,2016-03-31,42 days,top1 left
797,2016-07-01,2016-06-30,1 days,top1 left
2,2016-07-17,2016-06-30,17 days,top1 left
428,2016-09-01,2016-09-30,29 days,top1 left
905,2016-10-01,2016-09-30,1 days,top1 left


Clearly we have some dates that are way off, let's exclude them from our matches.

In [6]:
r = sj.gen_match_tables(1, top_limit=pd.Timedelta(days=10))
r['table left all']

,__left__,__right__,__diff__,__match type__
695,2015-10-01,2015-09-30,1 days,top1 left
747,2016-01-01,2015-12-31,1 days,top1 left
586,2016-04-01,2016-03-31,1 days,top1 left
797,2016-07-01,2016-06-30,1 days,top1 left
1,2016-09-30,2016-09-30,NaT,exact inner
905,2016-10-01,2016-09-30,1 days,top1 left
216,2016-10-09,2016-09-30,9 days,top1 left
955,2016-12-30,2016-12-31,1 days,top1 left
107,2017-01-01,2016-12-31,1 days,top1 left
160,2017-01-05,2016-12-31,5 days,top1 left


# Example 2: join 2 datasets with malformatted identifiers

In [7]:
df1=pd.read_csv('data/case_factors/securities.csv',parse_dates=['date'])
df2=pd.read_csv('data/case_factors/factors.csv',parse_dates=['Date'])

sj = join.smart_join.SmartJoin([df1, df2], [['BARRA_PIT_CUSIP','cusip'],['date','Date']], mode=['top1 left', 'top1 left'])

Let's run pre-join diagnostics. Again we would see the join would not be good, barely any of the identifiers matched! And none of the dates matched!

In [8]:
sj.stats_prejoin(False)


,key left,key right,all matched,left,right,inner,outer,unmatched total,unmatched left,unmatched right
0,BARRA_PIT_CUSIP,cusip,False,628,12692,0,13320,13320,628,12692
1,date,Date,False,2,2,1,3,2,1,1
2,__all__,__all__,False,1252,22975,0,24227,24227,1252,22975


Let's look at the closest matches for the identifiers and see what's going on: while the identifiers are almost the same, they are formatted differently. SmartJoin has automatically found the right identifier to match between the two different datasets.

In [14]:
r = sj.gen_match_tables(0, top_records=5)
r['table left'].head()

,__left__,__right__,__diff__,__match type__
2284,b'50188920',501889208,3,top1 left
31009,b'52466010',524660107,3,top1 left
13032,b'67066G10',67066G104,3,top1 left
40902,b'90431110',904311107,3,top1 left
56116,b'G5480U10',G5480U104,3,top1 left


Let's look at the closest matches for the dates and see what's going on: while the dates are almost the same, one is a BUSINESS date month-end while the other is a CALENDAR date month-end. Again, SmartJoin has automatically found the right matching date!

In [11]:
r = sj.gen_match_tables(1)

In [13]:
r['table left']

,__left__,__right__,__diff__,__match type__
1,2017-04-28,2017-04-30,2 days,top1 left
